<a href="https://colab.research.google.com/github/avilash/neural-networks-numpy/blob/master/Classification_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook trains a feed-forward neural network only in numpy to classify between the 10 digits in the MNIST Dataset

## Imports

In [0]:
import numpy as np
import matplotlib.pyplot as plt

## Model Forward and Backward

In [0]:
def act(x):
  return np.maximum(x, 0.)

In [0]:
def act_back(incoming_gradient, activation):
  incoming_gradient[activation == 0.] = 0.
  return incoming_gradient

In [0]:
def create_model(layer_sizes, input_size, output_size):
  model = []
  current_input_size = input_size
  for i, layer_size in enumerate(layer_sizes):
    layer = {}
    layer["W"] = np.random.randn(current_input_size, layer_size) / np.sqrt(current_input_size)
    layer["b"] = np.zeros(layer_size, dtype=np.float64)
    model.append(layer)
    current_input_size = layer_size
    
  layer = {}
  layer["W"] = np.random.randn(current_input_size, output_size) / np.sqrt(current_input_size)
  layer["b"] = np.zeros(output_size, dtype=np.float64)
  model.append(layer)

  return model

In [0]:
def forward(input, model):
  model_len = len(model)
    
  activations = []
  current_act = input
  activations.append(current_act)
 
  
  for i in range(model_len-1):
    layer = model[i]
    # print ("prev_A = {}, W = {}, B = {}".format(current_act.shape, layer['W'].shape, layer['b'].shape))
    current_act = act(np.matmul(current_act, layer['W']) + layer['b'])
    # print ("A = {}".format(current_act.shape))
    activations.append(current_act)

  layer = model[model_len-1]
  # print ("prev_A = {}, W = {}, B = {}".format(current_act.shape, layer['W'].shape, layer['b'].shape))
  pred = np.matmul(current_act, layer['W']) + layer['b']
  # print ("A = {}".format(pred.shape))
  activations.append(pred)
  
  
  return activations, pred

In [0]:
def backward(gradient, activations, model):
  gradients = []
  Z_grad = gradient.copy()
    
  activation_len = len(activations)
  
  for i in range(activation_len-2, -1, -1):
    grad = {}
    prev_A = activations[i]
    W = model[i]['W']
    grad['W'] = prev_A.T.dot(Z_grad)
    grad['b'] = np.sum(Z_grad, axis=0)
    grad['A'] = Z_grad.dot(W.T)
    gradients.insert(0, grad)
    Z_grad = grad['A'].copy()
    Z_grad = act_back(Z_grad, prev_A)
    
  return gradients
  

In [0]:
def probs_from_logits(logits):
  probs = np.exp(logits)
  probs = probs / (np.sum(probs, axis=1, keepdims=True) + np.finfo(float).eps)
  return probs

In [0]:
def ce_loss(y_pred, y):
  N = y_pred.shape[0]
  probs = probs_from_logits(y_pred)
  loss = -np.log(probs[np.arange(N), y])
  return loss

In [0]:
def ce_loss_back(y_pred, y):
  N = y_pred.shape[0]
  mask = np.zeros_like(y_pred)
  mask[np.arange(N), y] = 1.
  probs = probs_from_logits(y_pred)
  loss_back = probs - mask
  return loss_back

## Load DATA

### MNIST

Code adapted from : https://github.com/hsjeong5/MNIST-for-Numpy/blob/master/mnist.py

In [0]:
from urllib import request
import gzip
import pickle

mnist_filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in mnist_filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in mnist_filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in mnist_filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")
    
def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

In [12]:
download_mnist()
save_mnist()

Download complete.
Save complete.


In [0]:
x_train_full, y_train_full, x_test, y_test = load()
indices = np.random.permutation(x_train_full.shape[0])
val_idxs, train_idxs = indices[:10000], indices[10000:]
x_train, x_val = x_train_full[train_idxs,:], x_train_full[val_idxs,:]
y_train, y_val = y_train_full[train_idxs], y_train_full[val_idxs]

x_train , x_val, x_test = x_train.astype(np.float64) , x_val.astype(np.float64), x_test.astype(np.float64)
mean_image = np.mean(x_train, axis=0).reshape(1, x_train.shape[1])
x_train -= mean_image
x_val -= mean_image
x_test -= mean_image

## Training Procedure

### Weight Update

In [0]:
def update_weights(model, gradients, lr = 0.001):
  for i, grad in enumerate(gradients):
    model[i]['W'] -= lr*grad['W']
    model[i]['b'] -= lr*grad['b']
      
  return model

In [0]:
def init_rmsprop_params(model):
  rmsprop_params = []
  for layer in model:
    rmsprop_param = {}
    rmsprop_param['W'] = np.zeros_like(layer['W'])
    rmsprop_param['b'] = np.zeros_like(layer['b'])
    rmsprop_params.append(rmsprop_param)
  
  return rmsprop_params

In [0]:
def update_weights_rmsprop(model, gradients, rmsprop_params, lr = 0.001, beta=0.9):
  for i, grad in enumerate(gradients):
    v = beta*rmsprop_params[i]['W'] + (1.-beta)*np.square(grad['W'])
    model[i]['W'] -= (lr*grad['W'] / np.sqrt(v + 1e-14))
    rmsprop_params[i]['W'] = v
    
    v = beta*rmsprop_params[i]['b'] + (1.-beta)*np.square(grad['b'])
    model[i]['b'] -= (lr*grad['b'] / np.sqrt(v + 1e-14))
    rmsprop_params[i]['b'] = v
      
  return model, rmsprop_params

### Gradient Check

In [0]:
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [0]:
def grad_check(model, gradients, x_batch, y_batch):
  layer = 0
  layer_x = 0
  layer_y = 0
  grad_num = 0
  
  x_size, y_size = gradients[layer]['W'].shape
  for i in range(x_size):
    for j in range(y_size): 
      grad_num = gradients[layer]['W'][i][j]
      if grad_num != 0.:
        layer_x = i
        layer_y = j
        break
    if grad_num != 0.:
      break

  h = 0.00001
  oldval = model[layer]['W'][layer_x][layer_y]
  model[layer]['W'][layer_x][layer_y] = oldval - h
  _ , y_pred1 = forward(x_batch, model)
  loss1 = ce_loss(y_pred1, y_batch)
  model[layer]['W'][layer_x][layer_y] = oldval + h
  _ , y_pred2 = forward(x_batch, model)
  loss2 = ce_loss(y_pred2, y_batch)
  grad_ana = (loss2 - loss1) / (2.*h)
  grad_ana = np.sum(grad_ana)
  
  model[layer]['W'][layer_x][layer_y] = oldval

  error = rel_error(grad_num, grad_ana)
  return error

### Train / Test Loop

In [193]:
D_in, D_out = x_train.shape[1], 10
hidden_layers = [100]
model = create_model(hidden_layers, D_in, D_out)
rmsprop_params = init_rmsprop_params(model)
net_size = len(hidden_layers)

train_size = x_train.shape[0]
val_size = x_val.shape[0]

batch_start = 0
batch_end = 0

epochs = 10
batch_size = 64
num_batches = int(round(train_size / batch_size))

should_grad_check = True
if should_grad_check:
  x_batch = x_train[0:batch_size]
  y_batch = y_train[0:batch_size]
  activations, y_pred = forward(x_batch, model)
  loss = ce_loss(y_pred, y_batch).mean()
  loss_grad = ce_loss_back(y_pred, y_batch)
  gradients = backward(loss_grad, activations, model)
  err = grad_check(model, gradients, x_batch, y_batch)
  print ("Gradient Check - Rel Error = {} \n".format(err))

for epoch in range(epochs):
  loss = 0
  for i in range(num_batches):
    batch_start = (i%num_batches)*batch_size
    batch_end = min(train_size, batch_start+batch_size)
    x_batch = x_train[batch_start:batch_end]
    y_batch = y_train[batch_start:batch_end]

    activations, y_pred = forward(x_batch, model)

    loss += ce_loss(y_pred, y_batch).mean()

    loss_grad = ce_loss_back(y_pred, y_batch)

    gradients = backward(loss_grad, activations, model)
        
    model, rmsprop_params = update_weights_rmsprop(model, gradients, rmsprop_params, lr=0.0001)
        
    if (i+1)%100 == 0:
      print ("Epoch = {}, Batch = {}, Loss = {}".format(epoch+1, i+1, loss/100))
      loss = 0  
    
  _, train_pred = forward(x_train, model)
  train_acc = np.mean(np.argmax(train_pred, axis=1) == y_train)
  _, val_pred = forward(x_val, model)
  val_acc = np.mean(np.argmax(val_pred, axis=1) == y_val)
  print ("*** Summary ***")
  print ("Train Acc = {}, Val Acc = {}".format(train_acc, val_acc))
  print ("***************")

Gradient Check - Rel Error = 3.1573686448861186e-07 

Epoch = 1, Batch = 100, Loss = 25.23774951088449
Epoch = 1, Batch = 200, Loss = 6.695683753715192
Epoch = 1, Batch = 300, Loss = 4.722688101918565
Epoch = 1, Batch = 400, Loss = 4.041613333827541
Epoch = 1, Batch = 500, Loss = 3.4054990766856714
Epoch = 1, Batch = 600, Loss = 2.977395552583612
Epoch = 1, Batch = 700, Loss = 2.385888381965591
*** Summary ***
Train Acc = 0.90254, Val Acc = 0.8919
***************
Epoch = 2, Batch = 100, Loss = 2.3476988999884973
Epoch = 2, Batch = 200, Loss = 1.8982248709422092
Epoch = 2, Batch = 300, Loss = 1.7806526036859205
Epoch = 2, Batch = 400, Loss = 1.6290206430608414
Epoch = 2, Batch = 500, Loss = 1.5681206483870271
Epoch = 2, Batch = 600, Loss = 1.495096978874665
Epoch = 2, Batch = 700, Loss = 1.2421993890386662
*** Summary ***
Train Acc = 0.93288, Val Acc = 0.9133
***************
Epoch = 3, Batch = 100, Loss = 1.3659310341736335
Epoch = 3, Batch = 200, Loss = 1.1432392789792416
Epoch = 3, Ba

In [194]:
_, test_pred = forward(x_test, model)
test_acc = np.mean(np.argmax(test_pred, axis=1) == y_test)
print ("Test Acc = {}".format(test_acc))

Test Acc = 0.9433
